_See [Readme](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City#readme) for installation instructions_

---


In [ ]:
# Import libraries
import os.path 
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## If you use Notebook (and not JupyterLab), uncomment following lines
# import plotly.io as pio
# pio.renderers.default='notebook'


In [ ]:
# download ZIP and extract CSV
data_local_path = 'data/'
csv_filename = 'fr.openfoodfacts.org.products.csv'
csv_local_path = data_local_path+csv_filename

if not os.path.isfile(csv_local_path):
    zip_filename = csv_filename+'.zip'
    zip_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/'+zip_filename
    zip_local_path = data_local_path+zip_filename

    with urlopen(zip_url) as zip_resp:
        with ZipFile(BytesIO(zip_resp.read())) as zip_file:
            zip_file.extractall(data_local_path)


In [ ]:
# Deal with irregularities : 
# 23 data points are wrongly split into two lines : 
# - lines : 189070, 189105, 189111, 189121, 189154, 189162, 189164, 189170, 189244, 189246, 
#           189250, 189252, 189262, 189264, 189271, 189274, 189347, 189364, 189366, 189381, 
#           189406, 189408, 189419
# 
# The pattern is always the same : 
# - a NewLine character (`\n`) is placed at the end of column "first_packaging_code_geo" 
# - and the next line starts with a TAB separator (`\t`) : column "cities" is empty.
# 
# Since the first column ("code") is never empty, we just remove any `\n` character that is 
# directly followed by a TAB separator (`\t`).

clean_filename = 'fr.openfoodfacts.org.products-clean.csv'
clean_local_path = data_local_path+clean_filename

if not os.path.isfile(clean_local_path):
    with open(csv_local_path, 'r') as csv_file, open(clean_local_path, 'w') as clean_file:
        data = csv_file.read()
        clean_file.write(data.replace('\n\t', '\t'))


In [ ]:
# Read column names
column_names = pd.read_csv(clean_local_path, sep='\t', encoding='utf-8', nrows=0).columns.values

# Set column types according to fields description (https://static.openfoodfacts.org/data/data-fields.txt)
column_types = {col: 'Int64' for (col) in column_names if col.endswith(('_t', '_n'))}
column_types |= {col: float for (col) in column_names if col.endswith(('_100g', '_serving'))}
column_types |= {col: str for (col) in column_names if not col.endswith(('_t', '_n', '_100g', '_serving'))}

# Load raw data
raw_data = pd.read_csv(clean_local_path, sep='\t', encoding='utf-8',
     dtype=column_types,
     parse_dates=[col for (col) in column_names if col.endswith(('_datetime'))],
     infer_datetime_format=True,
     converters={
         
     }
)

# Display DataFrame size
raw_data.info()


In [ ]:
# Let's visualize columns emptiness
num_rows = len(raw_data.index)
columns_emptiness = pd.DataFrame({
    col : { 
        'count': raw_data[col].isna().sum(),
        'percent': 100 * raw_data[col].isna().sum() / num_rows,
    } for col in raw_data.columns
}).transpose().sort_values(by=['count'])

fig = px.bar(columns_emptiness,
    color='percent',
    y='percent',
    labels={
        'index':'column name',
        'percent':'% of empty values',
        'count':'# of empty values',
    },
    hover_data=['count'],
    title='Empty values per column',
    width=1200,
    height=800,
)
fig.show()


In [ ]:
# Let's keep only meaningful columns
meaningful_columns = ['product_name',
    'brands_tags', 'countries_tags',
    'additives_n', 
    'nutrition_grade_fr', 'nutrition-score-fr_100g',
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
    'fiber_100g', 'proteins_100g',
]
meaningful_data = raw_data[meaningful_columns]

# Display DataFrame size
meaningful_data.info()

# Display statistical summary of each column
meaningful_data.describe(include="all")


In [ ]:
# @TODO : supprimer les outliers

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(16,40))

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="nutrition-score-fr_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[0,0],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="additives_n",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[0,1],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="energy_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[1,0],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="saturated-fat_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[1,1],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="sugars_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[2,0],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="salt_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[2,1],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="fiber_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[3,0],
)

sns.boxplot(data=meaningful_data,
    x="nutrition_grade_fr", 
    y="proteins_100g",
    order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[3,1],
)


In [ ]:
sns.pairplot(data=meaningful_data[['nutrition_grade_fr', 'nutrition-score-fr_100g', 'energy_100g']], 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
)


In [ ]:

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="energy_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[0,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="saturated-fat_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[0,1],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="sugars_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[1,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="salt_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[1,1],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="fiber_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[2,0],
)

sns.jointplot(data=meaningful_data,
    x="nutrition-score-fr_100g", 
    y="proteins_100g", 
    hue="nutrition_grade_fr",
    hue_order=['a', 'b', 'c', 'd', 'e'],
    ax=axes[2,1],
)


In [ ]:

clean_data = raw_data.drop_duplicates()

# Display data types and empty values
clean_data.info()

clean_data.dropna(
        axis='columns',
        thresh=.1 * num_rows,
        inplace=True,
    )

# Display data types and empty values
clean_data.info()

num_cols = len(clean_data.columns)
clean_data.dropna(
        axis='index',
        thresh=.1 * num_cols,
        inplace=True,
    )

# Display data types and empty values
clean_data.info()

clean_data.drop_duplicates(inplace=True,)

# Display data types and empty values
clean_data.info()

# Display statistical summary of each column
clean_data.describe(include="all")


In [ ]:

fig = px.imshow(clean_data.isna(),
)
fig.show()


In [ ]:

meaningful_columns = ['product_name',
    'packaging_tags', 'brands_tags', 'manufacturing_places_tags', 'countries_tags',
    'main_category', 'categories_tags', 
    'labels_tags', 'additives_n', 'additives_tags', 
    'nutrition_grade_fr', 'nutrition-score-fr_100g',
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
    'fiber_100g', 'proteins_100g',
]


In [ ]:

# display value frequencies per column
for col in clean_data.columns:
    print(f'\n \
================================================\n \
>    { col }\n \
------------------------------------------------')
    display(clean_data[col].value_counts(dropna=False))


In [ ]:

raw_data = pd.read_csv(csv_local_path, 
    sep='\t',
    usecols=['product_name',
        'packaging_tags', 'brands_tags', 'manufacturing_places_tags', 'countries_tags',
        'main_category', 'categories_tags', 
        'labels_tags', 'additives_n', 'additives_tags', 
        'nutrition_grade_fr', 'nutrition-score-fr_100g',
        'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
        'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
    ])

# display first 5 rows
raw_data.head()


## Effectuer des opérations de nettoyage sur des données structurées



In [ ]:
# ❒ les éventuelles valeurs manquantes de chaque colonnes ont été identifiées, quantifiées et traitées



In [ ]:
# ❒ les lignes dupliquées ont été identifiées, quantifiées et traitées



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour nettoyer le jeu de données



In [ ]:
# ❒ une méthodologie de traitement des valeurs manquantes pour chaque colonne est justifiée et mise en oeuvre (ex : remplacer les valeurs manquantes d’une colonne par la valeur moyenne de la colonne)



In [ ]:
# ❒ une méthodologie de traitement des lignes dupliquées est justifiée et mise en oeuvre (ex : les lignes doublons ont été supprimés)  



In [ ]:
# ❒ les fonctionnalités d’édition de cellule Markdown du Jupyter Notebook sont utilisées dans au moins trois cellules pour décrire les choix méthodologiques et rendre lisible le document (titres, mise en forme, alternance de cellule d’exécution de code Python et de cellule de texte explicatif) 



In [ ]:
# ❒ la démarche de nettoyage des données est visible dans la structure du document (découpage du document en partie avec des titres clairs et mis en évidence, des commentaires à l’intérieur des parties pour expliciter la démarche, …)



## Effectuer une analyse statistique multivariée



In [ ]:
# ❒ au moins une méthode d’analyse descriptive est appliquée sur le jeu de données (ex : ACP)



In [ ]:
# ❒ au moins une méthode d’analyse explicative est appliquée sur le jeu de données (ex : ANOVA)



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour effectuer une analyse statistique multivariée



In [ ]:
# ❒ la méthode d’analyse descriptive appliquée sur le jeu de données est expliquée et justifiée



In [ ]:
# ❒ la méthode d’analyse explicative appliquée sur le jeu de données est expliquée et justifiée



## Communiquer ses résultats à l’aide de représentations graphiques lisibles et pertinentes



In [ ]:
# ❒ au moins trois types différents de graphiques ont été utilisés (ex : histogramme, boîte à moustache, nuage de points)



In [ ]:
# ❒ la justification des types de graphiques utilisés est explicitée dans le Jupyter Notebook.



In [ ]:
# ❒ au moins une fonction a été écrite, testée et utilisée pour effectuer une représentation graphique



In [ ]:
# ❒ les titres, valeurs des axes des abscisses et des ordonnées et légendes sont explicites



In [ ]:
# ❒ au moins un graphique interactif est utilisé pour illustrer une analyse lors de la présentation.



In [ ]:
# ❒ les titres, valeurs des axes des abscisses et des ordonnées et légendes sont affichés de manière lisible



---

_[Licence GPL-v3](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City/blob/main/LICENSE)_
